<a href="https://colab.research.google.com/github/casjunior93/Tera---Desafio-de-classificacao/blob/main/TERA_Desafio_de_Classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio de Detecção e Redução de Fraudes no Setor Financeiro**

O desafio é propor uma solução para detecção e análise de fraudes que possam reduzir os riscos da empresa de crédito em que trabalho e garantir margens saudáveis. 

Vou começar importando as bibliotecas iniciais:

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Agora importo os dados:

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Tera/DESAFIO CLASSIFICAÇÃO/fraud_detection_dataset.csv')

Vou retornar 10 linhas aleatórias para visualizar o dataframe:

In [3]:
df.sample(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6323505,688,CASH_IN,137386.78,C963022092,4727383.70,4864770.48,C848351329,295022.00,157635.22,0,0
5285450,372,PAYMENT,4225.10,C440199672,19959.00,15733.90,M418090682,0.00,0.00,0,0
5123205,356,CASH_IN,21880.95,C217885968,24483.00,46363.95,C1543763853,175404.93,153523.98,0,0
3472664,258,CASH_IN,109046.65,C532115428,8379791.29,8488837.93,C414451887,299591.41,190544.77,0,0
4588744,328,CASH_OUT,2948.65,C1737790926,0.00,0.00,C1195937175,141603.40,144552.05,0,0
309647,16,CASH_IN,246328.94,C1159771161,2122836.85,2369165.79,C1567520676,982367.06,829034.56,0,0
4132926,302,CASH_OUT,173094.66,C1641478698,0.00,0.00,C1468497001,383659.18,556753.84,0,0
3823657,282,CASH_OUT,65625.95,C714888703,88083.00,22457.05,C120109635,423748.29,489374.24,0,0
4053410,300,CASH_OUT,192885.66,C1382811312,0.00,0.00,C1181744111,245035.51,437921.17,0,0
1786884,162,CASH_IN,224797.18,C1706550671,149755.27,374552.45,C1479567915,268540.11,43742.94,0,0


## Conhecendo as features deste dataset:

* `step` - Passo: representa o total de horas transcorrido desde o início da simulação. Esta feature vai variar entre 1 e 744 (30 dias);

* `type` - Tipo: tipo de transação (depósito, saque, débito, pagamento e transferência);

* `amount` - Quantia: total que foi transacionado;

* `nameOrig` - ClienteOrigem: cliente que iniciou a transação

* `oldbalanceOrg` - SaldoInicialOrigem: saldo da conta de origem antes da transação;

* `newbalanceOrig` - SaldoFinalOrigem: saldo da conta de origem após a transação;

* `nameDest` - ClienteDestino: cliente de destino da transação;

* `oldbalanceDest` - SaldoInicialDestino: saldo da conta de destino antes da transação;

* `newbalanceDest` - SaldoFinalDestino: saldo da conta de destino após a transação;

* `isFraud` - ÉFraude: flag que define se a transação é fraudulenta ou não. Nesta simulação o objetivo da fraude é assumir a conta do usuário, esvaziá-la transferindo para outra conta e então sacando o dinheiro.

* `isFlaggedFraud` - SinalizadaComoFraude: automaticamente marcadas pelo banco como fraude por tentarem transferir mais de 200.000 em uma única transação.

# Pré-análise dos dados com SeetViz

Primeiro vou renomear as tabelas para facilitar a análise.

In [4]:
df = df.rename(
    columns={"step":"totalHoras", "type": "tipoTransacao", "amount": "quantia",
             "nameOrig": "clienteOrigem", "oldbalanceOrg": "saldoInicialOrigem",
             "newbalanceOrig": "saldoFinalOrigem", "nameDest": "clienteDestino",
             "oldbalanceDest": "saldoInicialDestino", "newbalanceDest": "saldoFinalDestino",
             "isFraud": "ehFraude", "isFlaggedFraud": "sinalizadaComoFraude"

             })
df.head(3)

,totalHoras,tipoTransacao,quantia,clienteOrigem,saldoInicialOrigem,saldoFinalOrigem,clienteDestino,saldoInicialDestino,saldoFinalDestino,ehFraude,sinalizadaComoFraude
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0


Agora importo o SweetViz

In [5]:
!pip install sweetviz
import sweetviz

O SeetViz gera um relatório em HTML com várias informações sobre as features do dataset.

In [6]:
analise=sweetviz.analyze(df)
analise.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Ao analisar o relatório do SweetViz, vi que o dataset tem 3 features categóricas (tipoTransacao, ehFraude e sinalizadaComoFraude), 6 numéricas (totalHoras, quantia, saldoInicialOrigem, saldoFinalOrigem, saldoInicialDestino e saldoFinalDestino) e 2 textuais (clienteOrigem, clienteDestino). Nenhuma feature tem valores faltantes.

# Tratamento de Dados

Vou tratar a feature categórica tipoTransacao com o método get_dummies do Pandas.

In [9]:
df.tipoTransacao.unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [10]:
df2 = pd.get_dummies(df.tipoTransacao)

A coluna tipoTransacao tem 5 valores únicos. Com o método get_dummies será criado uma coluna em um novo dataframe para cada valor único. Para cada transação, o tipo dessa transação vai receber o valor 1.

In [11]:
df2.head()

,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,1
3,0,1,0,0,0
4,0,0,0,1,0


Agora concateno o df2 com o dataframe original df.

In [15]:
df3 = pd.concat([df, df2], axis='columns')
df3.sample(10)

,totalHoras,tipoTransacao,quantia,clienteOrigem,saldoInicialOrigem,saldoFinalOrigem,clienteDestino,saldoInicialDestino,saldoFinalDestino,ehFraude,sinalizadaComoFraude,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
4412483,322,PAYMENT,754.90,C1355387169,11138.0,10383.10,M1740066436,0.00,0.00,0,0,0,0,0,1,0
2066865,181,CASH_OUT,287928.95,C1495910875,0.0,0.00,C1548802752,784324.93,1072253.88,0,0,0,1,0,0,0
4961443,351,PAYMENT,3676.76,C1415899602,0.0,0.00,M639091102,0.00,0.00,0,0,0,0,0,1,0
5251571,371,PAYMENT,18506.64,C643024443,103447.0,84940.36,M577751005,0.00,0.00,0,0,0,0,0,1,0
504576,20,PAYMENT,10622.33,C736858335,32082.0,21459.67,M1615016414,0.00,0.00,0,0,0,0,0,1,0
3227863,250,CASH_OUT,41535.73,C322373507,0.0,0.00,C2018751567,10023121.77,10064657.50,0,0,0,1,0,0,0
5948343,405,CASH_OUT,36280.58,C51992890,0.0,0.00,C979988731,3469527.07,3862897.21,0,0,0,1,0,0,0
4173821,304,CASH_OUT,45241.95,C620067240,0.0,0.00,C803628420,5992610.09,6037852.03,0,0,0,1,0,0,0
1325327,137,CASH_OUT,135913.97,C1791543984,3066.0,0.00,C1273230228,0.00,135913.97,0,0,0,1,0,0,0
2224420,186,PAYMENT,1459.74,C295290152,242330.0,240870.26,M1008058118,0.00,0.00,0,0,0,0,0,1,0


Agora vou dropar a coluna tipoTransacao.

In [21]:
df3.drop('tipoTransacao', axis='columns', inplace=True)
df3.sample(3)

,totalHoras,quantia,clienteOrigem,saldoInicialOrigem,saldoFinalOrigem,clienteDestino,saldoInicialDestino,saldoFinalDestino,ehFraude,sinalizadaComoFraude,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
2428547,202,319642.03,C339622965,0.00,0.00,C2042623141,364345.52,683987.55,0,0,0,1,0,0,0
1445489,140,84171.50,C936301146,9017429.55,9101601.06,C441077236,606674.13,522502.63,0,0,1,0,0,0,0
821154,40,87990.16,C1006696253,0.00,0.00,C192371454,7442860.10,7530850.26,0,0,0,1,0,0,0


Agora preciso tratar as features textuais.